# Analisi - Grafi

In [1]:
import pickle
import json

import geopandas as gpd
import networkx as nx
from networkx.algorithms.approximation import steiner_tree
import osmnx as ox

from my_paths import *
import Graph_functions

## Sport e tempo libero

Utilizzeremo i seguenti dati:
1. **Parchi**
2. **Impianti sportivi**
3. **Aree gioco**
Saranno i nostri "poi", cioè "Point of Interest" che verranno aggiunti al Grafo della rete Ciclabile/Stradale per la ricerca di percorsi

Carichiamo il Grafo di base della rete stradale/ciclabile

In [103]:
# Carichiamo Graph Ciclabili
with open(PATH_RETE_CICLABILE_COMPLETA_PICKLE_CLEAN, "rb") as f:
    G_strade_ciclabili = pickle.load(f)

In [104]:
G_compose = G_strade_ciclabili.copy()

### Aggiunta di **Parchi** come **"poi"**

1. Carichiamo il file dei parchi da "clean"
2. Utilizziamo **"connect_poi_nodes_to_graph()"** per connettere il geodataframe come "poi" al grafo delle strade/ciclabili.

In [105]:
gdf_parchi = gpd.read_file(PATH_PARCHI_CLEAN)
G_compose = Graph_functions.connect_poi_nodes_to_graph(G_compose, gdf_parchi,
                                                                     tipo = "parchi",
                                                                     poi = True,
                                                                     artificial=True)

### Aggiunta di **Impianti sportivi** come **"poi"**

In [106]:
gdf_impianti_sportivi = gpd.read_file(PATH_IMPIANTI_SPORTIVI_CLEAN)
G_compose = Graph_functions.connect_poi_nodes_to_graph(G_compose, gdf_impianti_sportivi,
                                                                     tipo = "impianti sportivi",
                                                                     poi = True,
                                                                     artificial=True)

### Aggiunta di **Aree gioco** come **"poi"**

In [107]:
gdf_aree_gioco = gpd.read_file(PATH_AREE_GIOCO_CLEAN)
G_compose = Graph_functions.connect_poi_nodes_to_graph(G_compose, gdf_aree_gioco,
                                                                     tipo = "aree gioco",
                                                                     poi = True,
                                                                     artificial=True)

## Utilizziamo algoritmo di **Steiner Tree**

Una volta aggiunti i "poi" al grafo delle strade/ciclabili, eseguiamo l'algoritmo che troverà i percorsi minimi per connettere tutti i "poi" tra di loro con il minor "peso" possibile.  
I pesi sono stati già accuratamente inseriti sugli archi delle strade, mentre quelli delle ciclabili e degli archi all'interno dei poligoni dei "poi" sono = 0, in modo da preferire sempre quei percorsi quando possibile (sfruttare ciclabili pre-esistenti il più possibile).

In [108]:
poi_nodes = [n for n, d in G_compose.nodes(data=True) if d.get("poi") == True]
G_sport_tempo_libero = nx.MultiDiGraph(steiner_tree(G_compose, terminal_nodes=poi_nodes, weight='weight')).to_undirected()
# Sembra ridontante castare a MultiDiGraph il risultato di steiner_tree() che è comunque un MultiDiGraph
# ma per vari motivi non si riesce poi a salvare in pickle e quindi meglio "rifare" una copia profonda in uno nuovo

Risultato lo salviamo in Clean di Analisi (per ora anche il pickle in staging) in geojson pronto da visualizzare

In [113]:
with open(PATH_SPORT_TEMPO_LIBERO_ANALISI_PICKLE_STAGING, "wb") as f:
    pickle.dump(G_sport_tempo_libero, f)

gdf_steiner = ox.graph_to_gdfs(G_sport_tempo_libero, edges=True, nodes=False).reset_index(drop=True)
gdf_steiner.to_file(PATH_SPORT_TEMPO_LIBERO_ANALISI_CLEAN, driver="GeoJSON")

## Analisi Automatica

Tramite la funzione Graph_functions.auto_analysis_poi() è possibile eseguire in automatico la ricreazione del grafo stradale con il dizionario dei pesi desiderato.
Si può ora caricaricare da file il dizionario dei pesi che abbiamo pre-stabilito.
Aggiunta possibilità di inserire come chiave "default" nel "custom_weights" in modo da dare un valore di base quando non viene trovata una strada. Grazie a questo è possibile utilizzare un dizionario del tipo:
```python
{"default": 1}
```
In questo modo a tutte le strade verrà assegnato il peso = 1.
Si possono inserire i vari percorsi di salvataggio dei file pickle (grafo) e geojson(geoDataFrame).
Richiede una struttura a liste di dizionari di geoDataFrame per "poi" del tipo:
```python
gdf_list = [
    {"gdf":  gpd.read_file(PATH_DEL_GEOJSON),
     "tipo": "tipo che si vuole assegnare agli archi che connettono il gdf al grafo",
     "attr": {"nome_attributo": "valore_attributo"} # Opzionale, di solito non ci serve
     }
]
```

In [ ]:
with open(PATH_CUSTOM_WEIGHTS_STRADE_RAW, 'r') as f:
    custom_weights = json.load(f)
gdf_list = [
    {"gdf":  gpd.read_file(PATH_PARCHI_CLEAN),
     "tipo": "parchi",},
    {"gdf":  gpd.read_file(PATH_IMPIANTI_SPORTIVI_CLEAN),
     "tipo": "impianti sportivi",},
    {"gdf":  gpd.read_file(PATH_AREE_GIOCO_CLEAN),
     "tipo": "aree gioco",}
]

Graph_functions.auto_analysis_poi(gdf_list, custom_weights,
                                  PATH_SPORT_TEMPO_LIBERO_ANALISI_CLEAN,
                                  PATH_SPORT_TEMPO_LIBERO_ANALISI_PICKLE_STAGING)

### Esempio di statisiche per confrontare 2 grafi: uno con i nostri pesi e l'altro fatto girare con pesi = 1

In [10]:
gdf_pesi = gpd.read_file(PATH_SPORT_TEMPO_LIBERO_ANALISI_CLEAN)
gdf_no_pesi = gpd.read_file("../Data/Clean/Analisi/sport_tempo_libero_clean_nopesi.geojson")

In [55]:
gdf_pesi_mod = gdf_pesi[gdf_pesi["artificiale"] != "True"]
gdf_no_pesi_mod = gdf_no_pesi[gdf_no_pesi["artificiale"] != "True"]

gdf_pesi_mod_most_highways = gdf_pesi_mod.to_crs(CRS_METR).groupby("highway").agg({
    "geometry": lambda x: sum(x.length/1000)
}).sort_values(by="geometry", ascending=False)[:5]
gdf_no_pesi_mod_most_highways = gdf_no_pesi_mod.to_crs(CRS_METR).groupby("highway").agg({
    "geometry": lambda x: sum(x.length/1000)
}).sort_values(by="geometry", ascending=False)[:5]

In [ ]:
print("----------PESI-------------")
print(gdf_pesi_mod_most_highways)
print("----------NO PESI----------")
print(gdf_no_pesi_mod_most_highways)